# Lets import required libraries

In [1]:
import requests 
import pandas as pd
from pandas import json_normalize
import json


# Extract json using requests library

In [2]:
path = "https://file.notion.so/f/s/98b0fa4b-2a85-40e0-af20-5a2785c55c92/Untitled.json?id=b6059ad3-5ee5-4f32-84b6-b03e68bdca98&table=block&spaceId=ca71608c-1cc3-4167-857a-24da97c78717&expirationTimestamp=1692856800000&signature=-4RtO-_UEk867fiOMecK_0DDHVmY_LsHZvJdHNH_1s0&downloadName=nested_data.json"
data = requests.get(path)
json_text = json.loads(data.text)

# load json file in a pandas DataFrame

In [3]:
df = pd.DataFrame(json_text)
normal_df = pd.json_normalize(df['programs'])
normal_df


,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37
3,1842-43,New York Philharmonic,"[{'Date': '1843-04-22T05:00:00Z', 'eventType':...",5887,"[{'workTitle': 'SYMPHONY NO. 2 IN D MAJOR, OP....",34ec2c2b-3297-4716-9831-b538310462b7
4,1843-44,New York Philharmonic,"[{'Date': '1843-11-18T05:00:00Z', 'eventType':...",305,"[{'workTitle': 'SYMPHONY NO. 7 IN A MAJOR, OP....",610a4acc-94e4-4cd6-bdc1-8ad020edc7e9
...,...,...,...,...,...,...
13949,2016-17,Musicians from the New York Philharmonic,"[{'Date': '2017-06-24T04:00:00Z', 'eventType':...",14189,"[{'workTitle': 'QUARTET, STRING, NO. 3 (MISHIM...",3d50968c-2e8b-405f-969c-ba36f941f393
13950,2016-17,New York Philharmonic,"[{'Date': '2017-07-02T04:00:00Z', 'eventType':...",14146,"[{'workTitle': 'SYMPHONY NO. 9, E MINOR, OP.95...",adb9fc8b-95bc-47d0-9f4b-78fc242f1f8f
13951,2016-17,New York Philharmonic,"[{'Date': '2017-07-03T04:00:00Z', 'eventType':...",14148,[{'workTitle': 'SYMPHONY NO. 7 IN E MINOR (SO...,f072c652-c452-4ad4-975a-f6abd5ff8b4d
13952,2016-17,New York Philharmonic,"[{'Date': '2017-07-04T04:00:00Z', 'eventType':...",14136,"[{'workTitle': 'SYMPHONY NO. 9, E MINOR, OP.95...",68625626-8085-44e4-bdb3-d8e6ac51c95b


# looks like two columns "works" and "concerts" have nested json files
# So, lets explode the list of both columns

In [4]:
a = normal_df.explode('concerts')
b = normal_df.explode('works')

# normalize the columns

In [5]:
concerts_df = pd.json_normalize(a['concerts'])
works_df = pd.json_normalize(b['works'])

# add columns to the exixting DataFrame

In [6]:
d = pd.concat([concerts_df, works_df, normal_df], axis=1)

In [7]:
final_df = d.drop(columns=['concerts', 'works'])

# looks like another column "soloists" has nested json

In [8]:
x = final_df.explode('soloists')

# lsts repeat the same process and store the output dataframe in a CSV file

In [9]:
soloists_df = pd.json_normalize(x['soloists'])

In [10]:
y = pd.concat([final_df, soloists_df], axis=1)

In [11]:
normalized_df = y.drop(columns=['soloists'])
normalized_df.to_csv("normalized_Dataframe.csv", index=False)

# DONE